<a href="https://colab.research.google.com/github/akshatshaw/Stock-sentiment-analysis/blob/main/basic_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
!pip install -q yfinance

In [ ]:
import yfinance as yf
import gensim

In [ ]:
df = pd.read_csv("/content/combined_stock_news_1.csv").drop(columns=['time','top_sentiment','sentiment_score','source'])

In [ ]:
df

,Unnamed: 0,date,title
0,0,2022-10-04,Don’t Miss the Turnaround in These 3 Stocks
1,1,2022-10-05,YMTC Ban Likely A Plus For Micron & Western Di...
2,2,2022-10-05,Advanced Micro Devices edges up even as worrie...
3,3,2022-10-06,"With earnings on tap, tech titans like Microso..."
4,4,2022-10-06,"Dear PSNY Stock Fans, Mark Your Calendars for ..."
...,...,...,...
31418,31418,2024-06-09,How to Play Apple’s Big AI Reveal Tomorrow
31419,31419,2024-06-10,Should You Jump on the META Stock Bandwagon? W...
31420,31420,2024-06-10,7 Blue-Chip Stocks to Add to Your June Buy List
31421,31421,2024-06-10,"Broadcom Could Report A Modest Beat For Q2, Sa..."


In [ ]:
df = df.groupby('date')['title'].agg(' '.join).reset_index()
df['Date']= pd.to_datetime(df['date'])
df.drop(columns=['date'], inplace = True)

In [ ]:
def process(data):
    data["Tomorrow"] = data["Close"].shift(-1)
    data["target"] = (data["Tomorrow"] > data["Close"]).astype(int)
    # data['P/L'] = data["Close"]-data['Open']
    data.drop(columns=['Volume','Adj Close'], inplace = True)
    return data

def pull_out(stock, data):
    price = pd.DataFrame(yf.download(stock, period="max"))
    # data = data[data.index>= '2023-12-31']
    price.reset_index(inplace = True)
    merged_df = pd.merge(data, price, on='Date', how='outer').dropna()
    return (merged_df)

In [ ]:
data = process(pull_out("NVDA", df))
#  = data['news_articles'].str.lower()
data['text'] = data.title.apply(gensim.utils.simple_preprocess)
# data['target'] =data['P/L'].apply(lambda x: 1 if x > 0 else 0)
news = data.copy()
data.to_csv("NVDA_v0.1.csv", index= False)
news.head()

[*********************100%%**********************]  1 of 1 completed


,title,Date,Open,High,Low,Close,Tomorrow,target,text
0,NASD Arbitration Panel Finds American Express ...,2005-02-22,0.23400,0.244083,0.231917,0.238583,0.30575,1,"[nasd, arbitration, panel, finds, american, ex..."
1,"American Express to cut 5,400 jobs, take charg...",2013-01-10,0.30800,0.309500,0.304000,0.305750,0.34125,1,"[american, express, to, cut, jobs, take, charg..."
2,American Express CFO Daniel Henry to retire,2013-05-07,0.34575,0.346500,0.338000,0.341250,0.36100,1,"[american, express, cfo, daniel, henry, to, re..."
3,Supreme Court rules for American Express in cl...,2013-06-20,0.36800,0.368500,0.358500,0.361000,0.36525,1,"[supreme, court, rules, for, american, express..."
4,American Express shares fall after Europe prop...,2013-07-17,0.36725,0.368500,0.364250,0.365250,0.39000,1,"[american, express, shares, fall, after, europ..."


In [ ]:
news = pd.read_csv("NVDA_v0.1.csv")

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(news.iterrows(), total=len(news)):
    text = row['title']
    myid = row['Date']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/1201 [00:00<?, ?it/s]

In [ ]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'Date'})
vaders = vaders.merge(news, how='left')

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.to("cuda")

In [ ]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
res = {}
for i, row in tqdm(news.iterrows(), total=len(news)):
    # try:
        text = row['title']
        myid = row['Date']
        vader_result = sia.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = polarity_scores_roberta(text)
        both = {**vader_result_rename, **roberta_result}
        res[myid] = vader_result_rename
    # except RuntimeError:
    #     print(f'Broke for id {myid}')

  0%|          | 0/1201 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'Date'})
results_df = results_df.merge(news, how='left').dropna()

In [ ]:
results_df.tail()

,Date,vader_neg,vader_neu,vader_pos,vader_compound,title,Open,High,Low,Close,Tomorrow,target,text
1195,2024-06-03,0.074,0.911,0.014,-0.9997,Immutep Collaborates With Merck To Evaluate Ef...,113.621002,115.000000,112.002998,115.000000,116.436996,1,"[immutep, collaborates, with, merck, to, evalu..."
1196,2024-06-04,0.010,0.871,0.119,1.0000,Ivy League Inspiration: 3 Stocks That Harvard’...,115.716003,116.599998,114.044998,116.436996,122.440002,1,"[ivy, league, inspiration, stocks, that, harva..."
1197,2024-06-05,0.003,0.835,0.162,1.0000,"Intel Stock (NASDAQ:INTC): It’s Game On, Even ...",118.371002,122.448997,117.468002,122.440002,120.998001,0,"[intel, stock, nasdaq, intc, it, game, on, eve..."
1198,2024-06-06,0.002,0.805,0.193,1.0000,"Dear Apple Stock Fans, Mark Your Calendars for...",124.047997,125.586998,118.320000,120.998001,120.888000,0,"[dear, apple, stock, fans, mark, your, calenda..."
1199,2024-06-07,0.002,0.799,0.200,1.0000,3 Underperforming Stocks Gearing for a Second ...,119.769997,121.692001,118.022003,120.888000,121.790001,1,"[underperforming, stocks, gearing, for, second..."


In [ ]:
X = results_df.drop(columns=['Date','Close','title','target', 'text','High','Low','Open','vader_pos'])
y=results_df['target']

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.plot(data['Open'])

In [ ]:
import gensim.downloader

glove_vectors = gensim.downloader.load('word2vec-google-news-300')


[================================================--] 96.2% 1599.9/1662.8MB downloaded

KeyboardInterrupt: 

In [ ]:
def get_average_word2vec(tokens, model):
    valid_tokens = [token for token in tokens if token in model]
    if not valid_tokens:
        return np.zeros(model.vector_size)
    vectors = [model[token] for token in valid_tokens]
    return np.mean(vectors, axis=0)

# Apply the function to the 'tokens' column to get the embeddings
news['embedding'] = news['text'].apply(lambda x: get_average_word2vec(x, glove_vectors))

In [ ]:
# Convert embeddings to separate columns
embedding_df = pd.DataFrame(news['embedding'].to_list(), index=news.index)
embedding_df.columns = [f'embedding_{i}' for i in range(embedding_df.shape[1])]

# Concatenate the embeddings with the original DataFrame
news = pd.concat([news, embedding_df], axis=1).drop(columns=['embedding'])

In [ ]:
# news.to_csv('final_nvda_v1.0.csv',index = False)

news = pd.read_csv('final_nvda_v1.0.csv')

In [ ]:
news.head(1)

,title,Date,Open,High,Low,Close,P/L,text,target,embedding_0,...,embedding_290,embedding_291,embedding_292,embedding_293,embedding_294,embedding_295,embedding_296,embedding_297,embedding_298,embedding_299
0,Don’t Miss the Turnaround in These 3 Stocks,2022-10-04,12.879,13.22,12.879,13.167,0.288,"['don', 'miss', 'the', 'turnaround', 'in', 'th...",1,0.008275,...,0.040109,-0.047701,-0.093689,-0.003278,-0.007533,-0.016087,0.000087,-0.01491,0.129647,0.014404


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range = (1,4),max_features= 10000)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size=0.2,random_state=2)

In [ ]:
# total = pd.DataFrame(tfidf.fit_transform(data['text']).toarray())
# train = pd.DataFrame(tfidf.fit_transform(X_train['text']).toarray())
# test= pd.DataFrame(tfidf.fit_transform(X_test['text']).toarray())

In [ ]:
# !pip install catboost

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,f1_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
bnb = BernoulliNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l2')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)
# cat = CatBoostClassifier( silent = True)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc,
    # 'NB': mnb,
    'BNB': bnb,
    # 'cat':cat,
#     'DT': dtc,
    # 'LR': lrc,
    # 'RF': rfc,
#     'AdaBoost': abc,
#     'BgC': bc,
#     'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
 }
from sklearn.model_selection import cross_val_score
def train_classifier(clf,X_train,y_train,X_test,y_test,total , y):
    csr_val = cross_val_score(clf, total, y, cv=3,scoring= "f1")
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    recall = recall_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    f1score = f1_score(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    return recall,precision ,f1score,accuracy, csr_val.mean()

for name,clf in clfs.items():

    recall,precision,f1score,accuracy,csr_val = train_classifier(clf, X_train,y_train,X_test,y_test,X , y)

    print("For ",name)
    print("recall - ",recall)
    print("Precision - ",precision)
    print("f1_score - ",f1score)
    print("accuracy - ",accuracy)
    print("cross_val_f1- ",csr_val)
    print("************************")

For  SVC
recall -  0.9492753623188406
Precision -  0.5695652173913044
f1_score -  0.7119565217391305
accuracy -  0.5583333333333333
cross_val_f1-  0.7240829346092504
************************
For  KN
recall -  0.6231884057971014
Precision -  0.5548387096774193
f1_score -  0.5870307167235496
accuracy -  0.49583333333333335
cross_val_f1-  0.44769404908136606
************************
For  BNB
recall -  1.0
Precision -  0.575
f1_score -  0.7301587301587301
accuracy -  0.575
cross_val_f1-  0.7240829346092504
************************
For  GBDT
recall -  0.8333333333333334
Precision -  0.5989583333333334
f1_score -  0.696969696969697
accuracy -  0.5833333333333334
cross_val_f1-  0.4293453305861179
************************
For  xgb
recall -  0.6304347826086957
Precision -  0.5686274509803921
f1_score -  0.5979381443298969
accuracy -  0.5125
cross_val_f1-  0.4684469882813766
************************


In [ ]:
X.columns =[i for i in range(303)]

##Try LSTM and BERT/ Transformer

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Define custom metrics
def recall_m(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

# Build the neural network model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),\
    layers.Dropout( 0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout( 0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout( 0.2),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model with custom metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', precision_m, recall_m, f1_m])

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc, test_precision, test_recall, test_f1 = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')
print(f'Test precision: {test_precision}')
print(f'Test recall: {test_recall}')
print(f'Test F1 score: {test_f1}')


Epoch 1/50
9/9 [==============================] - 2s 51ms/step - loss: 1.4250 - accuracy: 0.4735 - precision_m: 0.5128 - recall_m: 0.4606 - f1_m: 0.4526 - val_loss: 0.8515 - val_accuracy: 0.6061 - val_precision_m: 0.7292 - val_recall_m: 1.0000 - val_f1_m: 0.8249
Epoch 2/50
9/9 [==============================] - 0s 8ms/step - loss: 0.9585 - accuracy: 0.4659 - precision_m: 0.4983 - recall_m: 0.6381 - f1_m: 0.5574 - val_loss: 0.6683 - val_accuracy: 0.6061 - val_precision_m: 0.7292 - val_recall_m: 1.0000 - val_f1_m: 0.8249
Epoch 3/50
9/9 [==============================] - 0s 10ms/step - loss: 0.8502 - accuracy: 0.5114 - precision_m: 0.5401 - recall_m: 0.6346 - f1_m: 0.5712 - val_loss: 0.7114 - val_accuracy: 0.3939 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/50
9/9 [==============================] - 0s 9ms/step - loss: 0.7447 - accuracy: 0.5227 - precision_m: 0.5518 - recall_m: 0.7072 - f1_m: 0.6167 - val_loss: 0.7244 - val_accuracy: 0.3939 - val_